## TODO

## Data
#### Training dataset
- make a text-iterator for pubmed data
  - need to get pmids/dois for COVID-19 articles and filter those out
  - add, say, 60% of the text-data for COVID-19. 
  
#### Dev and test sets
- for dev and test, take 20% of covid-19 and add a random sample of pubmed approximately the same size.
- you might find that the sample of pubmed is not representative of real-world data, so you may have to have a larger sample with imbalanced classes.

## Modelling
- start with Doc2Vec and SVMs and see what you get. Then try a few other models and iterate. 

In [21]:
import pandas as pd
import numpy as np

In [22]:
pubmed = pd.read_csv('data/pubmed_sample.csv', dtype=str, index_col = 0)
covid = pd.read_csv('data/s2_cr_data.csv', dtype=str)
covid.shape, pubmed.shape

((12617, 14), (1020660, 9))

# Curious issue...
All of the pubmed titles end in '.', but this isn't true for CORD-19.

In [23]:
pubmed = pubmed.rename(columns = {'title':'venue','articletitle':'title'})

In [24]:
pubmed['title'] = [str(x)[:-1] if str(x)[-1]=='.' else str(x) for x in pubmed['title']]
covid['title'] = [str(x)[:-1] if str(x)[-1]=='.' else str(x) for x in covid['title']]

In [25]:

pubmed['tiabs'] = pubmed['title'] +'. '+pubmed['abstract']

In [26]:
pubmed.sample(5)

,venue,title,abstract,year,articledate,doi,pmid,pii,pmc,tiabs
573880,"Clinical biomechanics (Bristol, Avon)",Biomechanical analysis of motion following sac...,\nSacroiliac joint hypermobility or aberrant m...,2019,2019-05-25,10.1016/j.clinbiomech.2019.05.025,31234032,S0268-0033(18)30750-2,NaN,Biomechanical analysis of motion following sac...
532919,American journal of physiology. Renal physiology,Extracellular vesicles as a novel diagnostic a...,\nHypertension (HTN) affects one in three adul...,2019,2019-07-17,10.1152/ajprenal.00071.2019,31313949,NaN,PMC6766633,Extracellular vesicles as a novel diagnostic a...
975263,Food science & nutrition,Microbial quality of reduced-sodium napa cabba...,\nThis study evaluated the microbial safety of...,2019,2019-01-24,10.1002/fsn3.898,30847142,FSN3898,PMC6392845,Microbial quality of reduced-sodium napa cabba...
773427,Frontiers in chemistry,Novel 4-Acrylamido-Quinoline Derivatives as Po...,\nA novel structural series of quinoline deriv...,2019,2019-04-24,10.3389/fchem.2019.00236,31069214,NaN,PMC6491818,Novel 4-Acrylamido-Quinoline Derivatives as Po...
154758,Nephron,The Chronic Adverse Effect of Chloroquine on K...,"\nChloroquine (CQ), a classic autophagy inhibi...",2019,2019-10-29,10.1159/000503882,31661702,000503882,NaN,The Chronic Adverse Effect of Chloroquine on K...


Drop matching DOIs

In [27]:
covid.sample(5)

,pid,doi,title,abstract,authors,venue,year,tiabs,journal,journal-short,pubdate,issns,publisher,pre
2856,ee06cad1229867a6389ed8a7aaffb5148dd5f8ed,NaN,A*02:06-p4 (LTA-J-FLIFL) A*02:07-p5 (LLDS-J-YE...,A Supporting Information Figure 1 A. Design of...,NaN,NaN,NaN,A*02:06-p4 (LTA-J-FLIFL) A*02:07-p5 (LLDS-J-YE...,NaN,NaN,NaN,NaN,NaN,"['p', 'lta', 'j', 'flifl', 'p', 'lld', 'j', 'y..."
7168,3f9b24ea368416ce33708bb28327fb0b686b0ef0,10.1093/cid/ciy284,Counting Down the 2020 Goals for 9 Neglected T...,The control of neglected tropical diseases (NT...,"Hollingsworth, T",Clinical infectious diseases : an official pub...,2018,Counting Down the 2020 Goals for 9 Neglected T...,Clinical Infectious Diseases,NaN,2018-04-03,1058-4838; 1537-6591,Oxford University Press (OUP),"['count', 'goal', 'neglect_tropic_diseas', 'le..."
8417,d8a0e6a4cc626dfe6090ea49ce8b920eee3f98e1,NaN,Cohort profile: Studies of Work Environment an...,The aim of this article is to provide a detail...,"Id, Francesca; Julander, Anneli; Linde, Annika...",NaN,NaN,Cohort profile: Studies of Work Environment an...,NaN,NaN,NaN,NaN,NaN,"['cohort', 'profil', 'studi', 'work_environ', ..."
7272,6ccce7b6fb9a213959f56ae465bb75ad130d6c9a,10.1186/s13018-018-0836-8,Accuracy of MRI diagnosis of early osteonecros...,ObjectiveTo evaluate the overall diagnostic va...,"Zhang, Ya-Zhou; Cao, Xu-Yang; Li, Xi-Cheng; Ch...",Journal of Orthopaedic Surgery and Research,2018,Accuracy of MRI diagnosis of early osteonecros...,Journal of Orthopaedic Surgery and Research,J Orthop Surg Res,2018-07-04,1749-799X,Springer Science and Business Media LLC,"['accuraci', 'mri', 'diagnosi', 'earli_osteone..."
11263,1bed4f8a5412cfa315b0936705a647059ceaec62,NaN,nan,This virus is a positive sense single stranded...,NaN,NaN,NaN,This virus is a positive sense single stranded...,NaN,NaN,NaN,NaN,NaN,"['viru', 'posit_sens_singl_strand', 'rna', 'vi..."


In [28]:
covid_doi_set = set([str(x) for x in covid.doi.unique() if str(x) not in {'nan','None'}])

In [29]:
# drop any DOIs in both sets
pubmed = pubmed[~pubmed.doi.isin(covid_doi_set)]
pubmed.shape

(1019932, 10)

Drop matching titles

In [30]:
# try dropping matching titles.
covid_title_set = set([str(x) for x in covid.title.unique() if str(x) not in {'nan','None'}])
pubmed[~pubmed.title.isin(covid_title_set)].shape, len(list(covid_title_set))

((1019605, 10), 11989)

In [31]:
pubmed = pubmed[~pubmed.title.isin(covid_title_set)]
pubmed.shape, covid.shape

((1019605, 10), (12617, 14))

In [32]:
# also drop duplicates
pubmed = pubmed.drop_duplicates(subset = ['pmid','doi'], keep = 'first')
pubmed.shape

(1019598, 10)

Not a huge crossover - just a few hundred matching DOIs.  No PMIDs in the S2 API data as far as I can see. Title matching wasn't great - assumes perfect match. 

I'm concerned that we haven't filtered out enough covid-related content from the pubmed part of this to get a good classifier.

Consider using keywords to drop more content from pubmed.

In [33]:
covid = covid[~covid['tiabs'].isna()]
pubmed = pubmed[~pubmed['tiabs'].isna()]

## Build datasets

In [34]:
# add a response columne
covid['covid'] = 1
pubmed['covid'] = 0

In [35]:
# write out all data
# pd.concat([pubmed,covid]).to_csv('data/all_s2_pubmed.csv', encoding = 'utf-8-sig')

In [36]:
rs = 100

covid_train = covid.sample(int(0.6*covid.shape[0]), random_state = rs)
covid_devtest = covid[~covid['pid'].isin(set(covid_train['pid']))]
covid_dev = covid_devtest.sample(int(covid_devtest.shape[0]/2), random_state = rs)
covid_test = covid_devtest[~covid_devtest['pid'].isin(set(covid_dev['pid']))]
covid_train.shape, covid_dev.shape, covid_test.shape

((7570, 15), (2523, 15), (2524, 15))

In [37]:
pubmed_test = pubmed.sample(covid_dev.shape[0], random_state = rs)
pubmed = pubmed[~pubmed['pmid'].isin(set(pubmed_test['pmid']))]
pubmed_dev = pubmed.sample(covid_dev.shape[0], random_state = rs)
pubmed_train = pubmed[~pubmed['pmid'].isin(set(pubmed_dev['pmid']))]

pubmed_train.shape, pubmed_dev.shape, pubmed_test.shape

((884283, 11), (2523, 11), (2523, 11))

In [38]:
train = pd.concat([pubmed_train, covid_train])
test = pd.concat([pubmed_test, covid_test])
dev = pd.concat([pubmed_dev, covid_dev])

# randomize
train = train.sample(train.shape[0], random_state=rs)
test = test.sample(test.shape[0], random_state=rs)
dev = dev.sample(dev.shape[0], random_state=rs)

train.shape, test.shape, dev.shape

((891853, 19), (5047, 19), (5046, 19))

## Write out

In [39]:
train.to_csv('data/train.csv', encoding = 'utf-8-sig',index=False)
test.to_csv('data/test.csv', encoding = 'utf-8-sig',index=False)
dev.to_csv('data/dev.csv', encoding = 'utf-8-sig',index=False)